In [1]:
from helper import *
import warnings
warnings.filterwarnings('ignore')

import pandas as pd #a library to make the data more structured

In [2]:
fo = pd.ExcelFile('clean-data.xlsx') #read the data
df = pd.read_excel(fo, 'Sheet1') #create data frame

text = df['text']
text_list =  [i.split() for i in text]

In [28]:
print(len(text_list))


2390


In [29]:
#print(text_list)


In [30]:
#Create Bigram & Trigram Models 
from gensim.models import Phrases
# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
def engram(text_list, min_count=10):
    bigram = Phrases(text_list, min_count=10)
    trigram = Phrases(bigram[text_list])

    for idx in range(len(text_list)):
        for token in bigram[text_list[idx]]:
            #print(token)
            if '_' in token:
                #print(token)
                # Token is a bigram, add to document.
                text_list[idx].append(token)
        for token in trigram[text_list[idx]]:
            #print(token)
            if '_' in token:
                # Token is a bigram, add to document.
                text_list[idx].append(token)
    return text_list
text_list = engram(text_list, min_count=10)

In [38]:
#text_list



In [33]:
from gensim import corpora, models
def dictionariF(text_list):
    # Create a dictionary representation of the documents.
    dictionary = corpora.Dictionary(text_list)
    dictionary.filter_extremes(no_below=5, no_above=0.2) 
    #no_below (int, optional) – Keep tokens which are contained in at least no_below documents.
    #no_above (float, optional) – Keep tokens which are contained in no more than no_above documents (fraction of total corpus size, not an absolute number).

    return dictionary
dictionary = dictionariF(text_list)

In [40]:
#https://radimrehurek.com/gensim/tut1.html 
#build corpus

doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
#The function doc2bow converts document (a list of words) into the bag-of-words format

'''The function doc2bow() simply counts the number of occurrences of each distinct word, 
converts the word to its integer word id and returns the result as a sparse vector. 
The sparse vector [(0, 1), (1, 1)] therefore reads: in the document “Human computer interaction”, 
the words computer (id 0) and human (id 1) appear once; 
the other ten dictionary words appear (implicitly) zero times.'''

#print(len(doc_term_matrix))
#print(doc_term_matrix[100])

tfidf = models.TfidfModel(doc_term_matrix) #build TF-IDF model
corpus_tfidf = tfidf[doc_term_matrix]




In [9]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array
#function to compute coherence values
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, iterations=100)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

In [10]:
start=1
limit=21
step=1
model_list, coherence_values = compute_coherence_values(dictionary, corpus=corpus_tfidf, 
                                                        texts=text_list, start=start, limit=limit, step=step)
#show graphs
import matplotlib.pyplot as plt
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

<Figure size 640x480 with 1 Axes>

In [12]:
plt.show()

In [13]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 6))

Num Topics = 1  has Coherence Value of 0.601214
Num Topics = 2  has Coherence Value of 0.583027
Num Topics = 3  has Coherence Value of 0.596289
Num Topics = 4  has Coherence Value of 0.619571
Num Topics = 5  has Coherence Value of 0.619933
Num Topics = 6  has Coherence Value of 0.604157
Num Topics = 7  has Coherence Value of 0.617173
Num Topics = 8  has Coherence Value of 0.635649
Num Topics = 9  has Coherence Value of 0.614402
Num Topics = 10  has Coherence Value of 0.613387
Num Topics = 11  has Coherence Value of 0.635817
Num Topics = 12  has Coherence Value of 0.635174
Num Topics = 13  has Coherence Value of 0.607546
Num Topics = 14  has Coherence Value of 0.617774
Num Topics = 15  has Coherence Value of 0.618491
Num Topics = 16  has Coherence Value of 0.622155
Num Topics = 17  has Coherence Value of 0.603986
Num Topics = 18  has Coherence Value of 0.602456
Num Topics = 19  has Coherence Value of 0.604234
Num Topics = 20  has Coherence Value of 0.596159


In [42]:
from pprint import pprint

model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=4)
pprint(model.print_topics())

[(0,
  '0.076*"hadits" + 0.027*"hadits_palsu_hadits_palsu_hadits_palsu" + '
  '0.022*"hukum" + 0.022*"hadits_palsu_hadits_palsu" + 0.021*"hadits_palsu" + '
  '0.017*"puasa" + '
  '0.016*"hadits_palsu_hadits_palsu_hadits_palsu_hadits_palsu_hadits_palsu" + '
  '0.016*"cara" + 0.011*"shahih" + '
  '0.011*"hadits_palsu_hadits_palsu_hadits_palsu_hadits_palsu_hadits_palsu_hadits_palsu_hadits_palsu_hadits_palsu"'),
 (1,
  '0.057*"hukum" + 0.025*"islam" + 0.016*"fatwa_ulama_fatwa_ulama_fatwa_ulama" '
  '+ 0.015*"fatwa_ulama_fatwa_ulama" + 0.015*"wanita" + 0.015*"penjelasan" + '
  '0.015*"hari" + 0.014*"shalat" + 0.013*"syiah" + 0.012*"malam"'),
 (2,
  '0.055*"al_quran" + 0.038*"hadits_lemah_hadits_lemah_hadits_lemah" + '
  '0.033*"hadits_lemah_hadits_lemah" + 0.029*"hadits_lemah" + '
  '0.027*"hadits_lemah_hadits_lemah_hadits_lemah_hadits_lemah_hadits_lemah" + '
  '0.026*"shalat" + '
  '0.014*"hadits_lemah_hadits_lemah_hadits_lemah_hadits_lemah" + '
  '0.014*"hadits_lemah_hadits_lemah_hadits_l

In [50]:
topiknya = list()
for i in model[corpus_tfidf]:
    x = [i[0][1],i[1][1],i[2][1],i[3][1]]
    idenx_max_x = x.index(max(x))
    topiknya.append(idenx_max_x)
dictt = {
    "kode_topik":topiknya,
    "text":df['text'].tolist()
}

df = pd.DataFrame.from_dict(dictt)
df.to_excel("output.xlsx")

In [ ]:
komentar_baru = preprocessing(komentar_baru)
komentar_baru = ngram(komentar_baru)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in [komentar_baru]]
komentar_baru_tfidf = tfidf[doc_term_matrix]

def cek_topik(komentar_baru_tfidf):
    i = komentar_baru_tfidf
    x = [i[0][1],i[1][1],i[2][1],i[3][1]]
    idenx_max_x = x.index(max(x))
    return idenx_max_x

In [20]:
model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=15)

for idx, topic in model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.043*"zakat" + 0.038*"masjid" + 0.037*"tangan" + 0.034*"shalat" + 0.033*"fithri" + 0.028*"istri" + 0.027*"perayaan" + 0.024*"dekat" + 0.020*"hadits_palsu" + 0.020*"melihat"
Topic: 1 Word: 0.058*"keutamaan" + 0.041*"hari" + 0.039*"puasa" + 0.036*"hadits" + 0.033*"bulan" + 0.031*"al_quran" + 0.028*"tahun" + 0.027*"tiga" + 0.026*"perkara" + 0.026*"syubhat"
Topic: 2 Word: 0.048*"hadits_lemah" + 0.046*"makmum" + 0.045*"shalat_jumat" + 0.042*"shalat" + 0.034*"jumat" + 0.030*"imam" + 0.026*"tata_cara" + 0.026*"tata" + 0.026*"sikap" + 0.026*"pemerintah"
Topic: 3 Word: 0.071*"syariat" + 0.047*"golongan" + 0.041*"pakaian" + 0.041*"salah" + 0.040*"islam" + 0.040*"tetap" + 0.033*"pemimpin" + 0.019*"daging" + 0.018*"sunnah" + 0.017*"hukum"
Topic: 4 Word: 0.143*"hadits" + 0.055*"fatwa_ulama" + 0.040*"salaf" + 0.032*"hukum" + 0.029*"wa_sallam" + 0.028*"puasa" + 0.025*"fatwa" + 0.025*"syiah" + 0.024*"non_muslim" + 0.021*"sahabat"
Topic: 5 Word: 0.096*"hadits_dhaif" + 0.045*"dhaif" + 0.

In [19]:
for i in corpus_tfidf:
    print(len(i))

3
1
4
2
5
2
6
4
2
4
6
6
6
6
3
2
5
3
1
1
2
1
3
2
3
1
3
2
1
1
2
3
2
3
1
2
1
4
1
0
2
1
0
6
1
4
4
2
3
2
3
4
2
1
2
1
3
2
4
1
1
0
2
2
0
2
2
2
2
0
1
2
2
6
6
2
0
0
0
0
0
3
1
1
1
1
3
2
3
8
8
2
5
3
1
1
1
1
0
3
2
2
1
2
3
0
8
1
3
0
0
0
1
1
2
9
2
4
3
2
2
3
3
1
0
4
3
1
1
1
2
2
1
2
2
1
10
10
0
2
2
10
5
0
11
10
12
1
2
2
2
3
2
1
2
0
1
2
3
2
4
0
5
4
3
1
2
2
5
4
4
4
3
7
5
11
7
6
0
6
3
8
1
1
2
3
2
2
2
2
3
0
1
3
5
5
2
3
1
2
1
8
6
2
7
9
6
8
8
2
2
8
8
2
9
10
8
12
7
8
7
7
3
4
4
6
5
3
4
5
3
5
4
0
3
5
5
5
4
7
4
2
4
2
4
2
6
4
3
6
4
1
1
3
2
0
2
1
2
9
1
3
5
4
3
4
4
8
2
4
4
2
5
2
3
2
6
4
2
3
3
3
5
5
4
4
3
0
5
2
8
2
3
3
3
2
5
3
3
2
6
4
3
0
3
1
1
3
8
2
6
3
1
0
1
2
2
2
4
5
9
3
11
2
2
4
4
2
2
4
3
2
2
3
3
1
5
3
1
4
3
2
1
2
2
1
0
3
3
2
1
6
3
0
4
4
3
5
2
3
5
2
2
2
2
4
2
2
2
3
4
1
2
5
2
5
4
6
3
0
3
4
1
3
2
1
2
2
2
0
2
1
4
3
3
3
3
3
3
3
4
3
3
2
1
6
3
3
5
3
5
3
1
1
3
3
3
4
4
2
4
4
4
2
3
3
0
0
4
5
2
2
1
3
15
2
1
5
3
3
0
0
4
2
0
2
1
2
2
2
0
2
2
4
5
5
5
5
7
2
5
4
5
5
5
5
1
1
3
2
0
2
2
6
6
0
4
4
3
3
4
3
6
3
2
1
2
2
2
4
1
1
1
1
1

In [17]:
import pandas as pd
top_words_per_topic = []
for t in range(model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in model.show_topic(t, topn = 10)])

#pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words.csv")
df = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word','P']).to_csv("top_words2.csv")
print(df)

None


In [21]:
import gensim
import pyLDAvis.gensim;pyLDAvis.enable_notebook()

data = pyLDAvis.gensim.prepare(model, corpus_tfidf, dictionary)
print(data)
pyLDAvis.save_html(data, 'lda-gensim.html')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
4     -0.120354  0.244994       1        1  8.005549
5      0.011069  0.009175       2        1  7.656130
6      0.168941  0.020651       3        1  7.098236
10     0.082247  0.069603       4        1  7.092897
8     -0.195622 -0.090252       5        1  7.050929
1     -0.071335  0.096441       6        1  6.996595
9      0.102704 -0.058699       7        1  6.516655
13     0.092308  0.030336       8        1  6.513836
11     0.054198  0.005527       9        1  6.402919
12    -0.117338 -0.090671      10        1  6.347825
14     0.050057 -0.107936      11        1  6.270071
7     -0.018401  0.054089      12        1  6.162909
0      0.049187 -0.010678      13        1  6.085779
2      0.003681 -0.075301      14        1  5.987722
3     -0.091343 -0.097277      15        1  5.811958, topic_info=     Category        Freq                            Ter

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud as wd

for t in range(model.num_topics):
    plt.figure(figsize=(7,6))
    plt.imshow(wd(max_font_size=50, min_font_size=6).fit_words(dict(model.show_topic(t, 200))))
    plt.axis("off")
    plt.title("Topic #" + str(t))
    plt.savefig("wcld-topic-#"+str(t)+".png", facecolor='k', bbox_inches='tight')

plt.show()